# Análisis de costos de cursos en el mercado

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install hermetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%cd '/content/drive/MyDrive/Trabajo_de_Integracion_Curricular/Data-Mining-System-SETEC-main/3_data_preprocesing_module'

/content/drive/MyDrive/Trabajo_de_Integracion_Curricular/Data-Mining-System-SETEC-main/3_data_preprocesing_module


# Instalación e importación de librerías

In [4]:
import pandas as pd
import numpy as np
import os
from hermetrics.levenshtein import Levenshtein
import re

In [5]:
os.getcwd()

'/content/drive/MyDrive/Trabajo_de_Integracion_Curricular/Data-Mining-System-SETEC-main/3_data_preprocesing_module'

# Rutas de archivos

In [6]:
path = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_additional_data = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))
path_dwh = os.path.abspath(os.path.join(os.getcwd(),'DATAWAREHOUSE'))
path_ppd = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))

### Palabras clave

In [7]:
keyword = ['Python', 'programación web', 'paginas web', 'datos', 'Excel', 'Word', 'power point', 'BI', 'inteligencia de negocios', 'powerBI', 
'big data', 'programación', 'marketing', 'web', 'html', 'css',  'java', 'javascript', 'base de datos', 'redes', 'telecomunicaciones', 'Android', 'IOS', 'ARDUINO'
'Business Intelligence', 
'Desarrollo de software',
'Fibra Óptica',
'Gerencia de sistemas',
'Hackeo Ético',
'Informática Forense',
'Linux', 'WINDOWS'
'Office',
'PHP',
'Project',
'software R', 'Tableu', 'rapidminer', 
'Mysql', 'sql', 'sqlserver', 'Oracle', 'postgresql',
'Seguridad informática',
'Aulas virtuales', 'Moodle', 'software',
'Wordpress', 'joomla', 'xoops', 'drupal'
]

keyword = np.char.lower(np.array(keyword))

In [8]:
lev = Levenshtein()

In [9]:
stopwords = open(os.path.join(path_ppd,'spanish.txt'),"r", encoding='utf-8')
stopwords = stopwords.readlines()
stopwords = [x.replace('\n','') for x in stopwords]

In [10]:
def normalizar(text):
    puntuacion = '!"#$%&;\'()*+,-./?@[\]^_`{|}~()' 
    text = text.replace('Á','A')
    text = text.replace('É','E')
    text = text.replace('Í','I')
    text = text.replace('Ó','O')
    text = text.replace('Ú','U')
    text = text.strip().lower()
    text = text.split(' ')
    re_punc = re.compile('[%s]' % re.escape(puntuacion))
    stripped = [re_punc.sub(' ', w) for w in text]
    text = ' '.join(stripped)
    text = text.split(' ')
    new_text = []
    for i in range(len(text)):
        char = text[i]
        if char not in stopwords:
            new_text.append(text[i])
    new_text = ' '.join(new_text)
            
    return new_text

In [208]:
#Filtrar las areas de Tecnologia y TICs
df_dim_curso_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oc','dim_curso_perfil.csv'))
df_dim_curso_perfil = df_dim_curso_perfil[(df_dim_curso_perfil['area_familia']=='información y comunicación')|(df_dim_curso_perfil['area_familia'] == 'tecnologías de la información y comunicación')]
data_1= df_dim_curso_perfil.groupby(['curso_perfil','modalidad']).count().reset_index()
data_1 = data_1[['curso_perfil','modalidad']]

df_dim_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oec','dim_perfil.csv'))
df_dim_perfil = df_dim_perfil[df_dim_perfil['familia']=='tecnología: hardware y software (incluye tic)']
data_2 = df_dim_perfil.groupby('perfil').count().reset_index()
data_2['modalidad'] = 'desconocida'
data_2 = data_2[['perfil','modalidad']]

df_dim_curso = pd.read_csv(os.path.join(path_dwh,'datamart_ci','dim_curso.csv'))
df_dim_curso = df_dim_curso[df_dim_curso['area']=='tecnologías de la información y comunicación']
data_3 = df_dim_curso.groupby('curso').count().reset_index()
data_3['modalidad'] = 'desconocida'
data_3 = data_3[['curso','modalidad']]

arr_dims_curso_perfil = [df_dim_curso_perfil,df_dim_perfil,df_dim_curso]

In [216]:
data = np.concatenate((data_1.values,data_2.values,data_3.values),axis=0)
data = pd.DataFrame(data=data,columns=['curso_perfil','modalidad'])
data = data.drop_duplicates()
data['aux_2_curso_perfil'] = data['curso_perfil'].str.cat(data['modalidad'],sep=' | ')
data['aux_curso_perfil'] = data['curso_perfil'].apply(lambda x: normalizar(x))

In [217]:
df_coec = pd.read_csv(os.path.join(path_ppd,'costos_cursos_educacion_continua.csv'))
df_ccce = pd.read_csv(os.path.join(path_ppd,'costos_de_cursos_cec_epn.csv'))
df_coec = pd.concat([df_coec,df_ccce])
df_coec['aux_2_curso'] = df_coec['curso'].str.cat(df_coec['modalidad'],sep=' | ')
df_coec['aux_curso'] = df_coec['curso'].apply(lambda x: normalizar(x))

In [219]:
arr = []

for i in range(len(df_coec['aux_curso'])):
    # Similaridad 
    sub_arr = []
    for j in range(len(data['aux_curso_perfil'])):
        curso_fuente = df_coec['aux_curso'].iloc[i]
        curso_comparar = data['aux_curso_perfil'].iloc[j]
        similary = lev.similarity(curso_fuente,curso_comparar,(1,1.5,1.25))
        aux_similary=0
        arr_curso = curso_comparar.split(' ')
        if curso_comparar in curso_fuente :
            similary = 1.0
        else:
            for word in arr_curso:
                if word in curso_fuente:
                    aux_similary+=1/len(arr_curso)
            if data['modalidad'].iloc[j] == df_coec['modalidad'].iloc[i]:
              aux_similary += aux_similary/2
        if aux_similary > similary and aux_similary > 2/len(arr_curso):
            similary = aux_similary
        sub_arr.append(similary)
    arr.append(sub_arr)


In [220]:
res = pd.DataFrame(data=arr,index=df_coec['aux_2_curso'],columns=data['aux_2_curso_perfil'])

In [227]:

arr_dims_cols = ['curso_perfil','perfil','curso']
arr_datamarts = ['datamart_oc','datamart_oec','datamart_ci']
arr_files = ['dim_curso_perfil.csv','dim_perfil.csv','dim_curso.csv']
arr_df_final_dims = []
for i in range(len(arr_dims_curso_perfil)):
  df_dim = arr_dims_curso_perfil[i]
  df_dim['curso_modalidad'] = df_dim[arr_dims_cols[i]].str.cat(df_dim['modalidad'],sep=' | ')
  arr_curso_fuente = []
  for j in range(arr_dims_curso_perfil[i].shape[0]):
      query = df_dim['curso_modalidad'].iloc[j]
      val = res[res[query]==res[query].max()][query].index.values[0]
      arr_curso_fuente.append(df_coec[df_coec['aux_2_curso']==val][['curso','precio_por_hora']].values[0])
  df_curso_fuente_precio = pd.DataFrame(data=arr_curso_fuente,columns=['curso_fuente','precio_por_hora'])
  df_res_corr = pd.DataFrame(data=[df_dim[arr_dims_cols[i]].values,df_dim['modalidad'].values,df_curso_fuente_precio['curso_fuente'].values,df_curso_fuente_precio['precio_por_hora'].values])
  df_res_corr = df_res_corr.transpose()
  df_res_corr = df_res_corr.rename(columns={0:arr_dims_cols[i],1:'modalidad',2:'curso_fuente',3:'precio_por_hora'})
  df_dim = pd.read_csv(os.path.join(path_dwh,arr_datamarts[i],arr_files[i]))
  df_dim= pd.merge(df_dim,df_res_corr,left_on=[arr_dims_cols[i],'modalidad'],right_on=[arr_dims_cols[i],'modalidad'],how='left')
  df_dim['costo'] = df_dim['carga_horaria']*df_dim['precio_por_hora']
  df_dim = df_dim.drop(['curso_fuente','precio_por_hora'], axis=1)
  df_dim.to_csv(os.path.join(path_dwh,arr_datamarts[i],arr_files[i]),index=False)
  arr_df_final_dims.append(df_dim)
